In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Firstly, i am requesting for the wikipedia data

In [90]:
html_doc = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."
response = requests.get(html_doc)
print(response.status_code)


200


Then i search for the table i need 

In [91]:
soup = BeautifulSoup(response.text, 'html.parser')
table_contents=[]
indiatable=soup.find('table',{'class':"wikitable"})
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
df = df[df["Borough"] != "Not assigned"]
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                            'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                            'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [6]:
df.shape

(210, 3)

In [12]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I am merging the two different dataframes, with the postal codes that coincide

In [80]:
df1 = pd.merge(df, coordinates, left_on='Postcode', right_on='Postal Code')
df1.drop(["Postal Code"], axis=1, inplace=True)
df1.groupby(["Postcode"], axis=1)
df1

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
206,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
207,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
208,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999


And grouping the neighbours with the same postal code

In [89]:
df2 = df1.groupby(['Postcode','Borough','Latitude', 'Longitude'])['Neighbourhood'].apply(', '.join).reset_index()
df2

,Postcode,Borough,Latitude,Longitude,Neighbourhood
0,M1B,Scarborough,43.806686,-79.194353,"Rouge, Malvern"
1,M1C,Scarborough,43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,43.763573,-79.188711,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,43.770992,-79.216917,Woburn
4,M1H,Scarborough,43.773136,-79.239476,Cedarbrae
...,...,...,...,...,...
98,M9N,York,43.706876,-79.518188,Weston
99,M9P,Etobicoke,43.696319,-79.532242,Westmount
100,M9R,Etobicoke,43.688905,-79.554724,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,43.739416,-79.588437,"Albion Gardens, Beaumond Heights, Humbergate, ..."
